# [KTO]_Letter_analysis_v1_2210

**분석개요**

    A. 가볼래터 성과 분석 리포트
        1. 매체별 유입 분석
            1) 유입
            2) 재방문
            3) 구독 전환
            4) 이상 유저
            5) 앱설치
            6) 활성
            7) 리텐션
        2. 이동 경로 분석

In [1]:
# basic
import gc
import os
import sys
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#--------------------#
# handling
#--------------------#
import math
import time
import random
# import openpyxl
import importlib
import xlsxwriter
import numpy as np
import pandas as pd
from collections import Counter
from datetime import datetime, timedelta

#--------------------#
# Vis
#--------------------#
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
## Vis for jupyter theme
from IPython.display import Image
from jupyterthemes import jtplot
jtplot.style(theme= 'grade3', context='notebook', ticks=True, grid=False) ## dark backgroud jupyter notebook theme
# plt.style.use(['seaborn-white']) ## dark background style ## white style : 'seaborn-white'
plt.style.use(['dark_background']) ## dark background style ## white style : 'seaborn-white'
## Vis for korean 
import matplotlib.font_manager as fm
font_location = '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf'
fprop = fm.FontProperties(fname=font_location)
font_name = fprop.get_name()
matplotlib.rc('font', family=font_name)

In [40]:
#--------------------#
# sphere package
#--------------------#
sys.path.append("/home/das_share/sphere_class/")
import SpherePackage
from SpherePackage import *
for pkg in [SpherePackage] :
    _ = importlib.reload(pkg)

#--------------------#
# kto package
#--------------------#
sys.path.append("../src/")
import kto_config
import kto_util
import kto_prep
from kto_config import *
from kto_prep import *
from kto_util import *

for pkg in [kto_config, kto_prep, kto_util] :
    _ = importlib.reload(pkg)

# 1.Data Import

In [3]:
#------------------------------------------------#
# 1. log Data import
#------------------------------------------------#
# dates: 10월 한 달
df_app_log = DataImport.read_files(
    input_path = INPUT_PATH + DATA_TYPE + APP_KEY, 
    today = today + timedelta(days = 1), 
    dates = dates,
    platform_total = True, reduce_memory = True, verbose = False)

In [4]:
df_app_log_copy = df_app_log.copy()

# 2.Preprocess

In [5]:
#------------------------------------------------#
# 1. prep
#------------------------------------------------#
## 1) log
df_app_log = SpherePrep.BasicPrep.basic_prep(df_app_log)                # log data

## 2) user prop
df_prop = SpherePrep.Prop.df_prop_pipe(df_app_log, KEY_ID = [KEY_ID_DEVICE, KEY_ID_USER])       # user data
df_prop_manual = KtoPrep.import_prop_manul_data() 
df_prop = pd.merge(
    left = df_prop, right = df_prop_manual[[KEY_ID_USER, 'birth_y' #'gender'
    ]],
    on = KEY_ID_USER, how = 'left')

In [6]:
#------------------------------------------------#
# 1. prep
#------------------------------------------------#
## 3) param - 1
_, df_param_all = DataImport.json_to_dataframe_nodeN(
    df_app_log, [KEY_ID_DEVICE, KEY_ID_USER]
    )

In [7]:
#------------------------------------------------#
# 1. prep
#------------------------------------------------#
## 3) param - 2
_, df_param_all_session = DataImport.json_to_dataframe_nodeN( 
    df_app_log, [KEY_ID_DEVICE, KEY_ID_USER, 'sid']
    ) ### + param by session

In [8]:
#------------------------------------------------#
# 1. prep
#------------------------------------------------#
## 4) referr
lst_extrated_referr_cols = ['utm_campaign', 'utm_source', 'utm_medium']
df_app_log = SpherePrep.Attribution_Prep.gen_session_referr_cols(
        df_app_log, 
        referr_col = 'referrer', 
        lst_extrated_referr_cols = lst_extrated_referr_cols,
        fill_na_organic = True)

In [9]:
#------------------------------------------------#
# 2. param detail data
#------------------------------------------------#
## 1) event 기준
# df_param_install = DataImport.extract_event_param(df_param_all, EVENT_APP_INSTALL).drop_duplicates()
df_param_contents = DataImport.extract_event_param(df_param_all, EVENT_CONTENT_DETAIL).drop_duplicates()
df_param_subscribe = DataImport.extract_event_param(df_param_all, EVENT_SUBSCRIBE).drop_duplicates()
df_param_letter_main = DataImport.extract_event_param(df_param_all, EVENT_SUBSCRIBE_LETTER).drop_duplicates()
df_param_like = DataImport.extract_event_param(df_param_all, EVENT_SUBSCRIBE_LIKE).drop_duplicates()
df_param_signup = DataImport.extract_event_param(df_param_all, EVENT_SIGNUP).drop_duplicates()

## 2) 가볼래터 data 생성
df_param_contents_letter = df_param_contents[df_param_contents[PARAM_CONTENT_NAME].str.contains('가볼래-터', na=False)]

In [10]:
df_param_contents_letter[PARAM_CONTENT_NAME].unique()

array(['가볼래-터 구독하러 가기', "여행구독 '가볼래-터' 구독자 레벨업을 위한 ★미션 참여 방법& 선물 언박싱★",
       '＇가볼래-터＇구독 신청하고 미션 선물받자!',
       '[2차 앵콜] 다시 돌아온 가볼래-터 깜짝 이벤트! 여행구독 미션 참여하고, 카톡 이모티콘 받자!',
       '대한민국 구석구석 여행구독 서비스! 가볼래-터 구독자 대상 만족도 조사',
       '대한민국 구석구석 여행구독 서비스! 가볼래-터 구독자 대상 만족도 조사 당첨자 발표'], dtype=object)

## 1)Utils

In [13]:
## 원래 함수
def cnt_to_percent_ver1(df, col):
    df[col] = df[col].astype(float)

    if  df.index.dtype != 'str':
        df.index = df.index.astype('str')
    
    _lst = []
    for i in range(0,len(df)):
        x = round((df[col][i] / df[col].sum())*100, 2)
        _lst.append(x)

    df['percent']= _lst
    return df

# 3.Analysis
## 0)Segment Setting

In [12]:
#------------------------------------------------#
# 1. segment by referrer
#------------------------------------------------#
dict_lst_user_from_referrs = KtoPrep.get_user_list_by_referr(
    df_app_log, KEY_ID = KEY_ID_DEVICE,
    DEPTH_1_COL = 'referrer_utm_campaign', 
    DEPTH_2_COL = 'referrer_utm_source') 
dict_lst_user_from_referrs_gudok = dict_lst_user_from_referrs['2022_gudok']
dict_lst_user_from_referrs_gudok['organic']  = dict_lst_user_from_referrs['organic']['organic'] # organic 유입 추가

#------------------------------------------------#
# 2. segment by duration per device
#------------------------------------------------#
df_duration_per_device = df_app_log.groupby([KEY_ID_DEVICE])['duration'].sum()
dict_duration_seg = dict()
dict_duration_seg['high'] = df_duration_per_device[
    df_duration_per_device >= df_duration_per_device.quantile(0.66)].index
dict_duration_seg['middle'] = df_duration_per_device[
    (df_duration_per_device < df_duration_per_device.quantile(0.66)) &
    (df_duration_per_device >= df_duration_per_device.quantile(0.33))
    ].index
dict_duration_seg['low'] = df_duration_per_device[
    df_duration_per_device < df_duration_per_device.quantile(0.33)].index

#------------------------------------------------#
# 3. segment by new or re
#------------------------------------------------#
lst_new_device = list(df_app_log[df_app_log['created_date'] >= datetime.strptime(s_date, '%Y%m%d')][KEY_ID_DEVICE].unique())
lst_re_device = list(df_app_log[df_app_log['created_date'] < datetime.strptime(s_date, '%Y%m%d')][KEY_ID_DEVICE].unique())

#------------------------------------------------#
# 4. segment by fraud or total
#------------------------------------------------#
lst_fraud_device = list(df_app_log[df_app_log['duration'] <  5][KEY_ID_DEVICE].unique())
lst_total_device = list(df_app_log[KEY_ID_DEVICE].unique())

#------------------------------------------------#
# 5. segment by demo (gender or age)
#------------------------------------------------#
## gender
dict_seg_gender = dict()
dict_seg_gender['m'] = list(df_prop[df_prop['gender'] == 'm'][KEY_ID_DEVICE])
dict_seg_gender['f'] = list(df_prop[df_prop['gender'] == 'f'][KEY_ID_DEVICE])

## age
dict_seg_age = dict()
dict_seg_age['20_30'] = list(df_prop[(datetime.today().year - df_prop['birth_y']) < 40][KEY_ID_DEVICE])
dict_seg_age['40_50'] = list(df_prop[(datetime.today().year - df_prop['birth_y']) >= 40][KEY_ID_DEVICE])
# # ======================================================== #  next version
# dict_seg_age['20_30'] = list(df_prop[df_prop['age'] < 40][KEY_ID_DEVICE])
# dict_seg_age['40_50'] = list(df_prop[df_prop['age'] >= 40][KEY_ID_DEVICE])

#------------------------------------------------#
# 6. subscribe segment 
#------------------------------------------------#
lst_subscribe_device = list(df_app_log[
    df_app_log['abs_events'].apply(lambda x : True if EVENT_SUBSCRIBE in x else False)
    ][KEY_ID_DEVICE].values)

In [19]:
# prep: instagram, kakao_mo -> drop
del dict_lst_user_from_referrs_gudok['instagram']
del dict_lst_user_from_referrs_gudok['kakao_mo']
dict_lst_user_from_referrs_gudok.keys()

dict_keys(['kakaotalk_bizmessage', 'GDN', 'KAKAO_moment', 'FB_INSTA', 'NAVER_GFA', 'organic'])

## 1)매체 성과 분석
### (1)유입
- 전체 유입
- 신규/기존 유입

In [138]:
#########################################
# 1. 매체별 전체 유입인원
#########################################
dict_output_inflow_cnt = {}

for key, values in dict_lst_user_from_referrs_gudok.items():
    dict_output_inflow_cnt[f'{key}'] = {}
    dict_output_inflow_cnt[f'{key}']['inflow_cnt'] = len(values)

df_vis1 = pd.DataFrame.from_dict(dict_output_inflow_cnt, orient='index', columns=['inflow_cnt'])
df_vis1.sort_values(by = 'inflow_cnt', ascending=False)

# SphereVis.basic_bar_plot(
#     df_vis1, x = df_vis1.index, y = 'percent', title= '가볼래-터 매체별 전체 유입인원')
# plt.show()

,inflow_cnt
organic,1603771
kakaotalk_bizmessage,5327
NAVER_GFA,5102
GDN,4665
FB_INSTA,4447
KAKAO_moment,2868


In [166]:
#########################################
# 2. 신규/기존 유입
#########################################
## 1) 일별 전체 신규 유입
def get_daily_cnt(
                  dict_output, lst, nm, df_target = df_app_log, KEY_ID = KEY_ID_DEVICE, 
                  dict_input = dict_lst_user_from_referrs_gudok, make_sub_dict = False):
    # 1. 전체 일별 수치
    df_daily = df_app_log[df_app_log[KEY_ID].isin(lst)].groupby('day')[KEY_ID]\
                    .nunique().to_frame(name = f'total_{nm}')

    # 2. 매체별 일별 수지
    for key, values in dict_input.items():
        _lst_id = list(set(lst) &  set(values))
        _df = df_app_log[df_app_log[KEY_ID].isin(_lst_id)].groupby('day')[KEY_ID]\
                    .nunique().to_frame(name = f'{key}_{nm}').fillna(0)
        if make_sub_dict == True:
            dict_output[f'{key}'] = {}

        dict_output[f'{key}'][f'{nm}'] = _df
        # dict_output[f'{key}_{nm}'] 
        df_daily[f'{key}'] = _df[f'{key}_{nm}']
        df_daily.fillna(0, inplace=True)
        # pd.concat([df_daily, dict_output[f'{key}'][f'{key}_{nm}']], axis=1).fillna(0)
        
    return df_daily
    

df_daily_new_inflow = get_daily_cnt(dict_output_inflow_cnt, lst=lst_new_device, nm='new_inflow')
df_daily_new_inflow

## 3) 일별 전체 기존 유입
df_daily_re_inflow = get_daily_cnt(dict_output_inflow_cnt, lst=lst_re_device, nm='re_inflow')
df_daily_re_inflow

,total_new_inflow,kakaotalk_bizmessage,GDN,KAKAO_moment,FB_INSTA,NAVER_GFA,organic
day,,,,,,,
2022-10-01,93386,16,1.0,0.0,1.0,4.0,93138
2022-10-02,84310,8,1.0,0.0,0.0,2.0,84108
2022-10-03,61376,5,0.0,0.0,1.0,3.0,61195
2022-10-04,57880,9,1.0,0.0,2.0,1.0,57645
2022-10-05,55969,35,0.0,0.0,3.0,0.0,55618
2022-10-06,46015,10,0.0,0.0,0.0,3.0,45741
2022-10-07,69333,10,3.0,0.0,0.0,1.0,68930
2022-10-08,115547,11,1.0,1.0,0.0,5.0,115192
2022-10-09,61837,8,2.0,0.0,0.0,4.0,61554


,total_re_inflow,kakaotalk_bizmessage,GDN,KAKAO_moment,FB_INSTA,NAVER_GFA,organic
day,,,,,,,
2022-10-01,24470,47,0.0,0.0,1.0,3,24253
2022-10-02,20068,35,2.0,0.0,2.0,3,19877
2022-10-03,13907,25,6.0,0.0,1.0,3,13689
2022-10-04,13604,39,3.0,0.0,1.0,1,13389
2022-10-05,13117,142,1.0,0.0,3.0,1,12842
2022-10-06,13224,41,2.0,1.0,0.0,5,13031
2022-10-07,16509,43,2.0,2.0,2.0,4,16206
2022-10-08,20795,20,3.0,1.0,1.0,3,20600
2022-10-09,14770,13,4.0,0.0,3.0,5,14604


### (2)재방문
- 방문 빈도별 재방문율
- 주차별 재방문율

In [142]:
#########################################
# 1. 방문 빈도별 재방문율
#########################################
## data setting by referrs
dict_visit_cnt = {}
dict_df_param_contents = {}

for key, values in dict_lst_user_from_referrs_gudok.items():
    dict_df_param_contents[key] = df_param_contents_letter[
                                        df_param_contents_letter[KEY_ID_DEVICE].isin(
                                            dict_lst_user_from_referrs_gudok[key])]
                                            
    _df = dict_df_param_contents[key].groupby(KEY_ID_DEVICE)['day'].size().to_frame(
        name = 'day_cnt')
        
    VISIT_1_CNT = len(_df[_df['day_cnt'] == 1])
    VISIT_2_CNT = len(_df[_df['day_cnt'] == 2])
    VISIT_3_CNT = len(_df[_df['day_cnt'] >= 3])

    VISIT_1_RATE = VISIT_1_CNT / len(_df)
    REVISIT_2_RATE= 1 - VISIT_1_RATE
    REVISIT_3_RATE = 1 - ((VISIT_1_CNT + VISIT_2_CNT) / len(_df))

    # dict_visit_cnt[f'{key}'] = {}
    dict_visit_cnt[f'{key}']= [VISIT_1_RATE, REVISIT_2_RATE, REVISIT_3_RATE]

df_vis_revisit = pd.DataFrame.from_dict(
    dict_visit_cnt, orient='index', columns=[
        'only_1_visit', 'revisit_rate', 'revisit_over_3_rate'])

df_vis_revisit

,only_1_visit,revisit_rate,revisit_over_3_rate
kakaotalk_bizmessage,0.366081,0.633919,0.416236
GDN,0.823085,0.176915,0.068393
KAKAO_moment,0.948029,0.051971,0.011161
FB_INSTA,0.849078,0.150922,0.057355
NAVER_GFA,0.867503,0.132497,0.057232
organic,0.657959,0.342041,0.204999


In [41]:
#########################################
# 2. 주차별 재방문율
#########################################
## 1) 전체 주차별 재방문율
df_retention = SphereRetention.retention_main(
    df_app_log, KEY_ID = KEY_ID_DEVICE, period = 'week', vis = True, title = 'Total Retention')

## 2) 매체별 주차별 재방문율
for key, values in dict_lst_user_from_referrs_gudok.items():
    df_retention = SphereRetention.retention_main(
        df_app_log[df_app_log[KEY_ID_DEVICE].isin(values)], 
        KEY_ID = KEY_ID_DEVICE, period = 'week', vis = True, title = f'{key} Retention')

TypeError: retention_main() got an unexpected keyword argument 'title'

### (3)구독전환
- 구독전환수
- 구독전환율

In [172]:
#########################################
# 1. 구독완료
#########################################
## 1) 일별 구독완료 유저 수
dict_subscribe = {}

df_daily_subscribe = pd.DataFrame()
df_daily_subscribe = get_daily_cnt(dict_output=dict_subscribe, lst=lst_total_device, nm='subscribe_cnt', 
                    make_sub_dict = True)
df_daily_subscribe

,total_subscribe_cnt,kakaotalk_bizmessage,GDN,KAKAO_moment,FB_INSTA,NAVER_GFA,organic
day,,,,,,,
2022-10-01,117856,63,1.0,0.0,2.0,7,117391
2022-10-02,104378,43,3.0,0.0,2.0,5,103985
2022-10-03,75283,30,6.0,0.0,2.0,6,74884
2022-10-04,71484,48,4.0,0.0,3.0,2,71034
2022-10-05,69086,177,1.0,0.0,6.0,1,68460
2022-10-06,59239,51,2.0,1.0,0.0,8,58772
2022-10-07,85842,53,5.0,2.0,2.0,5,85136
2022-10-08,136342,31,4.0,2.0,1.0,8,135792
2022-10-09,76607,21,6.0,0.0,3.0,9,76158


In [174]:
#########################################
# 2. 구독전환율
#########################################
## 매체별 전환율
### 랜딩페이지 > 구독
df_subscribe_conversion = pd.DataFrame(
                            index=[
                                'landing_cnt', 
                                'subscribe_cnt', 
                                'subscribe_conversion'])

for key, df in dict_df_param_contents.items():
    _df = df[df[PARAM_CONTENT_NAME] == '가볼래-터 구독하러 가기']
    LANDING_CNT = _df[KEY_ID_DEVICE].nunique()

    _df = df_param_subscribe[
                        df_param_subscribe[KEY_ID_DEVICE].isin(
                            list(_df[KEY_ID_DEVICE])
                        )]
    SUBSCRIBE_CNT = _df[KEY_ID_DEVICE].nunique()
    dict_subscribe[f'{key}']['subscribe_conversion'] = [LANDING_CNT,
                                                    SUBSCRIBE_CNT, 
                                                    (SUBSCRIBE_CNT/LANDING_CNT)]

    df_subscribe_conversion[f'{key}'] =\
        dict_subscribe[f'{key}']['subscribe_conversion']

df_subscribe_conversion

,kakaotalk_bizmessage,GDN,KAKAO_moment,FB_INSTA,NAVER_GFA,organic
landing_cnt,1649.000000,4633.000000,2867.0,4446.00000,5101.000000,22810.000000
subscribe_cnt,50.000000,5.000000,0.0,59.00000,26.000000,1569.000000
subscribe_conversion,0.030321,0.001079,0.0,0.01327,0.005097,0.068786


### (4)Fraud
- 바운스율
- 정착율

In [44]:
#########################################
# 1. 바운스 유저 비율 & 정착 유저 비율
#########################################
dict_fraud_referrs = {}
for key, values in dict_lst_user_from_referrs_gudok.items():
    _df = df_app_log[df_app_log[KEY_ID_DEVICE].isin(values)]
    _df = _df.groupby(KEY_ID_DEVICE)['duration'].sum().reset_index()
    lst_device_fraud = list(_df[KEY_ID_DEVICE][_df['duration'] < 5].unique())

    TOTAL_CNT = _df[KEY_ID_DEVICE].nunique()
    FRAUD_CNT = len(lst_device_fraud)
    FRAUD_RATE = FRAUD_CNT / TOTAL_CNT
    # SETTLE_RATE = 1 - FRAUD_RATE

    dict_fraud_referrs[f'{key}'] = [TOTAL_CNT, FRAUD_CNT, FRAUD_RATE,  1 - FRAUD_RATE]

df_fraud_referrs = pd.DataFrame.from_dict(
    dict_fraud_referrs, orient='index', columns=[
        'total_cnt', 'fraud_cnt', 'fraud_rate', 'settle_rate'])

df_fraud_referrs

,total_cnt,fraud_cnt,fraud_rate,settle_rate
kakaotalk_bizmessage,5327,682,0.128027,0.871973
GDN,4665,3821,0.819078,0.180922
KAKAO_moment,2868,2717,0.947350,0.052650
FB_INSTA,4447,3789,0.852035,0.147965
NAVER_GFA,5102,4537,0.889259,0.110741
organic,1603771,1154811,0.720060,0.279940


### (5)앱설치 유저

In [30]:
#########################################
# 0. prep
#########################################
df_id_prep = SpherePrep.Prop.df_prop_pipe(
                df_app_log, [KEY_ID_DEVICE, KEY_ID_USER], 
                drop_none_prop = True)

#########################################
# 1. 앱설치 유저
#########################################
dict_install = {}
df_install_cnt = pd.DataFrame()
_df_log_target = df_app_log[df_app_log['abs_events'].apply(lambda x : True if EVENT_APP_INSTALL in x else False)]

for key, lst in dict_lst_user_from_referrs_gudok.items():
    _lst = list(set(df_id_prep[KEY_ID_DEVICE][df_id_prep[KEY_ID_DEVICE].isin(lst)]))

    dict_install[f'{key}'] = {}
    dict_install[f'{key}']['device_id'] =\
        _df_log_target[KEY_ID_DEVICE][_df_log_target[KEY_ID_DEVICE].isin(_lst)]

    dict_install[f'{key}']['cnt'] = len(dict_install[f'{key}']['device_id'])
    df_install_cnt[f'{key}_cnt'] = dict_install[f'{key}']['cnt']

    print(f'{key}: ', dict_install[f'{key}']['cnt'])

kakaotalk_bizmessage:  0
GDN:  0
KAKAO_moment:  0
FB_INSTA:  0
NAVER_GFA:  0
organic:  4303


In [32]:
# # 2. gender & age_group 
# ## KAKAO_moment: drop (설치자 0명이라서)
# dict_lst_user_from_referrs_gudok_demo = dict_lst_user_from_referrs_gudok
# del dict_lst_user_from_referrs_gudok_demo['KAKAO_moment']

# ## 시각화
# Profiling.demo_profiling(
#     df_user_prop=df_prop, KEY_ID=KEY_ID_DEVICE, 
#     dict_user_list=dict_lst_user_from_referrs_gudok_demo, 
#     total_avg = True, vis_type = 'all')

In [31]:
# ## exact numbers age/gender by referrs
# _df_gender = pd.DataFrame()
# _df_age = pd.DataFrame()

# for key, lst_id in dict_lst_user_from_referrs_gudok_demo.items():
#     if key != 'total':
#         _df = df_prop[df_prop[KEY_ID_DEVICE].isin(lst_id)]
#         _df_gender = pd.concat(
#             [_df_gender, _df['gender'].value_counts().to_frame(name = f'{key}_gender')], axis=1).fillna(0)
#         _df_age = pd.concat(
#             [_df_age, _df['age_group'].value_counts().to_frame(name = f'{key}_age')], axis=1).fillna(0)
#     else:
#         pass

# _df_gender
# _df_age

### (6)활성
- 인당 체류 시간
- 인당 컨텐츠 조회수
- 검색 조회 빈도

In [77]:
#########################################
# 1. 인당 체류시간
#########################################
lst_device_no_fraud = list(set(lst_total_device) - set(lst_device_fraud))
_df_log_target = df_app_log[df_app_log[KEY_ID_DEVICE].isin(lst_device_no_fraud)]

dict_duration = {}
for key, values in dict_lst_user_from_referrs_gudok.items():
    _df = _df_log_target[_df_log_target[KEY_ID_DEVICE].isin(values)]

    dict_duration[f'{key}'] = {}
    dict_duration[f'{key}']['duration'] = _df.groupby(
                        KEY_ID_DEVICE)['duration'].sum().to_frame(
                            name = 'total_duration')

    print(f'{key} 인당 평균 체류시간: ', dict_duration[f'{key}']['duration']['total_duration'].mean())

kakaotalk_bizmessage 인당 평균 체류시간:  226.0729469825155
GDN 인당 평균 체류시간:  69.2335439914163
KAKAO_moment 인당 평균 체류시간:  18.509108079748163
FB_INSTA 인당 평균 체류시간:  18.878477298396206
NAVER_GFA 인당 평균 체류시간:  36.337178797468354
organic 인당 평균 체류시간:  267.205120171062


In [87]:
#########################################
# 2. 인당 컨텐츠 조회수
#########################################
## 1) 가볼래-터 랜딩페이지 접속 유저 한정
### 이상 유저 제거 x
_lst_id = df_param_contents[KEY_ID_DEVICE][
    df_param_contents[PARAM_CONTENT_NAME]=='가볼래-터 구독하러 가기']

_df_log_target = df_app_log[df_app_log[KEY_ID_DEVICE].isin(_lst_id)]

_df_log_target = df_app_log[
    df_app_log['abs_events'].apply(lambda x : True
                                    if ('balanceGameMain' in x) or 
                                    ('balanceGame' in x) or 
                                    ('travelLetterDetail' in x) or
                                    ('travelLetterLike' in x) or 
                                    ('travelLetterNotLike' in x) else False)]

dict_page_cnt = {}
for key, values in dict_lst_user_from_referrs_gudok.items():
    _df = _df_log_target[_df_log_target[KEY_ID_DEVICE].isin(values)]
    _series_event_cnt = _df['abs_events'].apply(lambda x : len(x))
    _df['page_cnt'] = _series_event_cnt
    
    dict_page_cnt[f'{key}'] = _series_event_cnt
    print(f'{key} 인당 평균 컨텐츠 조회수: ', round(_df['page_cnt'].mean(), 2))

### 이상 유저 제외
# _df_log_target_prep = _df_log_target[_df_log_target[KEY_ID_DEVICE].isin(lst_device_no_fraud)]


kakaotalk_bizmessage 인당 평균 컨텐츠 조회수:  5.82
GDN 인당 평균 컨텐츠 조회수:  4.25
KAKAO_moment 인당 평균 컨텐츠 조회수:  20.67
FB_INSTA 인당 평균 컨텐츠 조회수:  17.88
NAVER_GFA 인당 평균 컨텐츠 조회수:  19.5
organic 인당 평균 컨텐츠 조회수:  13.8


## 2)이동경로